In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
m = 100000

x = np.random.random(size=m)
X = x.reshape(-1,1)

In [5]:
y = 4. * x + 3. + np.random.normal(0, 3, size=m)

In [6]:
X.shape

(100000, 1)

In [17]:
def J(theta, x_b, y):
    try:
        return np.sum((y - x_b.dot(theta)) ** 2) / len(y)
    except:
        return float('inf')

def DJ(theta, x_b, y):
    return x_b.T.dot(x_b.dot(theta) - y) * 2 / len(y)

def gradient_descent(x_b, y, int_theta, eta, n_iters=1e4, epsilon=1e-8):
    theta = int_theta
    i_ters = 0
    
    while i_ters < n_iters:
        gradient = DJ(theta, x_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        
        if (abs(J(theta, x_b, y) - J(last_theta, x_b, y)) < epsilon):
            break
        i_ters += 1
    
    return theta

In [15]:
%%time
X_b = np.hstack([np.ones((len(X), 1)), X]) # 为x添加一列
ini_theta = np.zeros(X_b.shape[1])
eta = 0.01
theta = gradient_descent(X_b, y, ini_theta, eta)
theta

Wall time: 30.8 s


In [18]:
X_b = np.hstack([np.ones((len(X), 1)), X]) # 为x添加一列
ini_theta = np.zeros(X_b.shape[1])
eta = 0.01
theta = gradient_descent(X_b, y, ini_theta, eta)
theta

array([3.00360224, 4.01283393])

In [19]:
def DJ_sgd(theta, x_b_i, y_i):
    return x_b_i.T.dot(x_b_i.dot(theta) - y_i) * 2

In [28]:
def sgd(X_b, y, int_theta, n_iters=1e4):
    
    t0 = 5
    t1 = 50
    
    def learning_rate(t):
        return t0 / (t + t1)
    
    theta = int_theta
    
    for cur_inter in range(n_iters):
        rand_i = np.random.randint(len(X_b))
        gradient = DJ_sgd(theta, X_b[rand_i], y[rand_i])
        theta = theta - learning_rate(cur_inter) * gradient
    
    return theta

In [29]:
%%time
X_b = np.hstack([np.ones((len(X), 1)), X]) # 为x添加一列
ini_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b, y, ini_theta, n_iters=len(X_b)//3)

Wall time: 598 ms


In [30]:
X_b = np.hstack([np.ones((len(X), 1)), X]) # 为x添加一列
ini_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b, y, ini_theta, n_iters=len(X_b)//3)

In [31]:
theta

array([2.95171417, 4.07894838])

In [32]:
X_b = np.hstack([np.ones((len(X), 1)), X]) # 为x添加一列
ini_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b, y, ini_theta, n_iters=len(X_b))
theta

array([3.01681474, 3.98502281])

In [69]:
class my_LinearRegnession:
    
    def __init__(self):
        self.conf_ = None
        self.itercept_ = None
        self._theta = None
        
    def fit_normal(self, x_train, y_train):
        assert x_train.shape[0] == y_train.shape[0], \
            "the size"
        X_b = np.hstack([np.ones((len(x_train), 1)),x_train])
        self._theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_train)  # np.linalg.inv 求逆
        self.itercept_ = self._theta[0]
        self.conf_ = self._theta[1:]
        
        return self
    
    def fit_gb(self, x_train, y_train):
        assert x_train.shape[0] == y_train.shape[0], \
            "the size"
        def J(theta, x_b, y):
            try:
                return np.sum((y - x_b.dot(theta)) ** 2) / len(y)
            except:
                return float('inf')
        
        def DJ(theta, x_b, y):
            res = np.empty(len(theta))

            res[0] = np.sum(x_b.dot(theta) - y)

            for i in range(1, len(theta)):
                res[i] = (x_b.dot(theta) - y).dot(x_b[:, i])

            return res * 2 / len(x_b)
        
        def gradient_descent(x_b, y, int_theta, eta, n_iters=1e4, epsilon=1e-8):
            theta = int_theta
            i_ters = 0

            while i_ters < n_iters:
                gradient = DJ(theta, x_b, y)
                last_theta = theta
                theta = theta - eta * gradient

                if (abs(J(theta, x_b, y) - J(last_theta, x_b, y)) < epsilon):
                    break
                i_ters += 1

            return theta
        
        X_b = np.hstack([np.ones((len(X), 1)), X]) # 为x添加一列
        ini_theta = np.zeros(X_b.shape[1])
        self._theta = gradient_descent(X_b, y,ini_theta, eta)
        self.itercept_ = self._theta[0]
        self.conf_ = self._theta[1:]
        
        return self
    
    def fit_sgd(self, X_train, y_train, n_iters=5, t0=5, t1=50):
        assert X_train.shape[0] == y_train.shape[0], \
            "the size"
        assert n_iters >= 1, "n_iters > 0"
        
        def DJ_sgd(theta, X_b_i, y_i):
            return X_b_i.T.dot(X_b_i.dot(theta) - y_i) * 2
        
        def sgd(X_b, y, int_theta, n_iters=1e4, t0=5, t1=50):
    
            def learning_rate(t):
                return t0 / (t + t1)

            theta = int_theta
            m = len(X_b)

            for cur_inter in range(n_iters):
                indexs = np.random.permutation(m)
                X_b_new = X_b[indexs]
                y_new = y[indexs]
                for i in range(m):
                    gradient = DJ_sgd(theta, X_b_new[i], y_new[i])
                    theta = theta - learning_rate(cur_inter * m + i) * gradient

            return theta
        
        X_b = np.hstack([np.ones((len(X_train), 1)), X_train]) # 为x添加一列
        ini_theta = np.random.randn(X_b.shape[1])
        self._theta = sgd(X_b, y_train, ini_theta, n_iters, t0, t1)
        self.itercept_ = self._theta[0]
        self.conf_ = self._theta[1:]
        
        return self
    
    def predict(self, x_predict):
        assert self.itercept_ is not None and self.conf_ is not None, \
            "fit before"
        assert x_predict.shape[1] == len(self.conf_), \
            "number "
        X_b = np.hstack([np.ones((len(x_predict), 1)), x_predict])
        
        return X_b.dot(self._theta)
    
    def my_mean_squared_error(self, y_true, y_predict):
        return np.sum((y_predict - y_true) ** 2) / len(y_true)
    
    def my_r2_score(self, y_true, y_predict):
        return 1 - self.my_mean_squared_error(y_true, y_predict) / np.var(y_true)
 
    def score(self, x_test, y_test):
        y_redict = self.predict(x_test)
        return self.my_r2_score(y_test, y_redict)
    
    def __repr__(self):
        return "my_LinearRegnession()"

### 使用自己的SGD

In [70]:
m = 100000

x = np.random.random(size=m)
X = x.reshape(-1,1)
y = 4. * x + 3. + np.random.normal(0, 3, size=m)

In [71]:
lin_reg = my_LinearRegnession()

In [72]:
lin_reg.fit_sgd(X, y, n_iters=2)

my_LinearRegnession()

In [73]:
lin_reg.conf_

array([3.96546316])

In [74]:
lin_reg.itercept_

3.0129855701325763

### 使用真实的数据

In [75]:
from sklearn import datasets

In [76]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [79]:
from sklearn.preprocessing import StandardScaler
standarScaler = StandardScaler()
standarScaler.fit(x_train)
x_train_stand = standarScaler.transform(x_train)
x_test_stand = standarScaler.transform(x_test)

In [103]:
lin_reg1 = my_LinearRegnession()
lin_reg1.fit_sgd(x_train_stand, y_train, n_iters=2)
lin_reg1.score(x_test_stand, y_test)

0.7812439799187596

In [113]:
lin_reg1 = my_LinearRegnession()
lin_reg1.fit_sgd(x_train_stand, y_train, n_iters=50)
lin_reg1.score(x_test_stand, y_test)

0.8131998413030975

### sklearn 中的SGD

In [114]:
from sklearn.linear_model import SGDRegressor

In [115]:
sgd_reg = SGDRegressor()

In [116]:
%time sgd_reg.fit(x_train_stand, y_train)
sgd_reg.score(x_test_stand, y_test)

Wall time: 19 ms


d:\Users\wzy\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.8034735378672317

In [117]:
sgd_reg = SGDRegressor(n_iter=100)
%time sgd_reg.fit(x_train_stand, y_train)
sgd_reg.score(x_test_stand, y_test)

Wall time: 9.99 ms


d:\Users\wzy\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.8133723789110521